In [1]:
import json
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
tqdm.pandas()

/home/soroosh/.local/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
file = 'data/result.json'
data = json.load(open(file))
data.keys()

dict_keys(['name', 'type', 'id', 'messages'])

In [3]:
messages = data['messages']

In [4]:
import re

def clean_text(text):
    valid_chars = '1234567890آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهیي ؟!۱۲۳۴۵۶۷۸۹۰'
#     return re.sub('[^{}]'.format(valid_chars), '', text)
    return text

def parse_text(text):
    if isinstance(text, str):
        return clean_text(text) + ' '
    
    all_types = ['bank_card', 'bold', 'code', 'email',
                 'hashtag', 'italic', 'link', 'mention',
                 'phone', 'text_link','underline']
    valid_types = ['bold']
    res = ''
    for t in text:
        if isinstance(t, str):
            res += clean_text(t) + ' '
        elif t['type'] in valid_types:
            res += clean_text(t['text']) + ' '
    return res

def get_sender(sender):
    return 'soroush' if sender.lower() == 'soroush' else 'erf'

In [5]:
df = pd.DataFrame(data=messages)
df = df[['from', 'text']]

In [6]:
df.drop(df[pd.isna(df['from'])].index, inplace=True)

In [7]:
df['from'] = df.progress_apply(lambda x: get_sender(x['from']), axis=1)

In [8]:
df['text'] = df.progress_apply(lambda x: parse_text(x['text']).replace('  ',' ').strip(), axis=1)

In [9]:
df.drop(df[df['text'] == ''].index, inplace=True)
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
print(df.shape)

(183978, 2)


In [108]:
history = [{'from':df.loc[0]['from'] , 'text': df.loc[0]['text']}]

In [109]:
for f,t in tqdm(zip(df['from'][1:], df['text'][1:])):
    if f == history[-1]['from']:
        history[-1]['text'] += ' ' + t
    else:
        history += [{'from':f, 'text': t}]

In [110]:
df2 = pd.DataFrame(data=history)

In [111]:
df2.shape

(91831, 2)

In [113]:
df2.to_csv('data/processed_cleaned.csv', index=False)

In [11]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')

In [12]:
tok.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [17]:
tok.decode(tok.encode(df['text'][1000]))

'مگه سيبريه؟'